In [1]:
!pip install -qU oikan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [2]:
!pip install -qU pmlb

In [3]:
import numpy as np
import torch
import time
import tracemalloc
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, accuracy_score, precision_score, f1_score
from pmlb import fetch_data
from oikan import OIKANRegressor, OIKANClassifier

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Determine device (CPU or GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def adjust_augmentation_factor(n_samples, augmentation_factor=10, min_augmented_size=1000, max_augmented_size=100000):
    augmented_size = augmentation_factor * n_samples
    if augmented_size < min_augmented_size:
        augmentation_factor = max(augmentation_factor, (min_augmented_size + n_samples - 1) // n_samples)
    elif augmented_size > max_augmented_size:
        augmentation_factor = max(1, max_augmented_size // n_samples)
    return augmentation_factor


In [5]:
def benchmark_regression_dataset(dataset_name, counter, total):
    """Benchmark OIKANRegressor and ElasticNet on a regression dataset."""
    try:
        # Fetch data
        data = fetch_data(dataset_name)
        X = data.drop('target', axis=1).values
        y = data['target'].values
        
        # Check dataset size (approximate memory usage in bytes)
        memory_usage = X.nbytes + y.nbytes
        if memory_usage > 600 * 1024 * 1024:  # 600MB in bytes
            print(f"Skipping dataset {dataset_name} due to size exceeding 600MB.")
            return None
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Adjust augmentation factor
        augmentation_factor = adjust_augmentation_factor(len(X_train))
        num_augmented_rows = augmentation_factor * len(X_train)
        num_features = X.shape[1]
        
        # Check augmented size (approximate, assuming float64 for simplicity)
        augmented_memory = num_augmented_rows * (num_features + 1) * 8  # 8 bytes per float64
        if augmented_memory > 600 * 1024 * 1024:
            augmentation_factor = max(1, (600 * 1024 * 1024) // (len(X_train) * (num_features + 1) * 8))
            num_augmented_rows = augmentation_factor * len(X_train)
        
        # Initialize OIKAN model
        oikan_model = OIKANRegressor(augmentation_factor=augmentation_factor, verbose=False)
        
        # Training benchmark for OIKAN
        tracemalloc.start()
        start_time = time.time()
        oikan_model.fit(X_train, y_train)
        train_time = time.time() - start_time
        _, train_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Prediction benchmark for OIKAN
        tracemalloc.start()
        start_time = time.time()
        y_pred_oikan = oikan_model.predict(X_test)
        predict_time = time.time() - start_time
        _, predict_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Metrics for OIKAN
        rmse_oikan = np.sqrt(mean_squared_error(y_test, y_pred_oikan))
        r2_oikan = r2_score(y_test, y_pred_oikan)
        mape_oikan = mean_absolute_percentage_error(y_test, y_pred_oikan)
        
        # ElasticNet model
        elasticnet_model = ElasticNet(random_state=42)
        elasticnet_model.fit(X_train, y_train)
        y_pred_elasticnet = elasticnet_model.predict(X_test)
        
        # Metrics for ElasticNet
        rmse_elasticnet = np.sqrt(mean_squared_error(y_test, y_pred_elasticnet))
        r2_elasticnet = r2_score(y_test, y_pred_elasticnet)
        mape_elasticnet = mean_absolute_percentage_error(y_test, y_pred_elasticnet)
        
        # Print progress
        print(f"Regression dataset: {dataset_name} - completed ({counter} / {total}) ✅")
        
        return {
            'dataset': dataset_name,
            'device': device,
            'num_features': num_features,
            'num_augmented_rows': num_augmented_rows,
            'augmentation_factor': augmentation_factor,
            'train_time': train_time,
            'train_memory': train_memory / 10**6,  # Convert to MB
            'predict_time': predict_time,
            'predict_memory': predict_memory / 10**6,  # Convert to MB
            'rmse_oikan': rmse_oikan,
            'r2_oikan': r2_oikan,
            'mape_oikan': mape_oikan,
            'rmse_elasticnet': rmse_elasticnet,
            'r2_elasticnet': r2_elasticnet,
            'mape_elasticnet': mape_elasticnet
        }
    except Exception as e:
        print(f"Error benchmarking regression dataset {dataset_name}: {e}")
        return None

In [6]:
def benchmark_classification_dataset(dataset_name, counter, total):
    """Benchmark OIKANClassifier and LogisticRegression (ElasticNet equivalent) on a classification dataset."""
    try:
        # Fetch data
        data = fetch_data(dataset_name)
        X = data.drop('target', axis=1).values
        y = data['target'].values
        
        # Check dataset size (approximate memory usage in bytes)
        memory_usage = X.nbytes + y.nbytes
        if memory_usage > 600 * 1024 * 1024:  # 600MB in bytes
            print(f"Skipping dataset {dataset_name} due to size exceeding 600MB.")
            return None
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Adjust augmentation factor
        augmentation_factor = adjust_augmentation_factor(len(X_train))
        num_augmented_rows = augmentation_factor * len(X_train)
        num_features = X.shape[1]
        
        # Check augmented size (approximate)
        augmented_memory = num_augmented_rows * (num_features + 1) * 8
        if augmented_memory > 600 * 1024 * 1024:
            augmentation_factor = max(1, (600 * 1024 * 1024) // (len(X_train) * (num_features + 1) * 8))
            num_augmented_rows = augmentation_factor * len(X_train)
        
        # Initialize OIKAN model
        oikan_model = OIKANClassifier(augmentation_factor=augmentation_factor, verbose=False)
        
        # Training benchmark for OIKAN
        tracemalloc.start()
        start_time = time.time()
        oikan_model.fit(X_train, y_train)
        train_time = time.time() - start_time
        _, train_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Prediction benchmark for OIKAN
        tracemalloc.start()
        start_time = time.time()
        y_pred_oikan = oikan_model.predict(X_test)
        predict_time = time.time() - start_time
        _, predict_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        # Metrics for OIKAN
        accuracy_oikan = accuracy_score(y_test, y_pred_oikan)
        precision_oikan = precision_score(y_test, y_pred_oikan, average='weighted', zero_division=0)
        f1_oikan = f1_score(y_test, y_pred_oikan, average='weighted', zero_division=0)
        
        # ElasticNet equivalent (using LogisticRegression with elasticnet penalty)
        elasticnet_model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, random_state=42, max_iter=1000)
        elasticnet_model.fit(X_train, y_train)
        y_pred_elasticnet = elasticnet_model.predict(X_test)
        
        # Metrics for ElasticNet
        accuracy_elasticnet = accuracy_score(y_test, y_pred_elasticnet)
        precision_elasticnet = precision_score(y_test, y_pred_elasticnet, average='weighted', zero_division=0)
        f1_elasticnet = f1_score(y_test, y_pred_elasticnet, average='weighted', zero_division=0)
        
        # Print progress
        print(f"Classification dataset: {dataset_name} - completed ({counter} / {total}) ✅")
        
        return {
            'dataset': dataset_name,
            'device': device,
            'num_features': num_features,
            'num_augmented_rows': num_augmented_rows,
            'augmentation_factor': augmentation_factor,
            'train_time': train_time,
            'train_memory': train_memory / 10**6,  # Convert to MB
            'predict_time': predict_time,
            'predict_memory': predict_memory / 10**6,  # Convert to MB
            'accuracy_oikan': accuracy_oikan,
            'precision_oikan': precision_oikan,
            'f1_oikan': f1_oikan,
            'accuracy_elasticnet': accuracy_elasticnet,
            'precision_elasticnet': precision_elasticnet,
            'f1_elasticnet': f1_elasticnet
        }
    except Exception as e:
        print(f"Error benchmarking classification dataset {dataset_name}: {e}")
        return None

In [7]:
def calculate_summary_stats(df, task):
    """Calculate AVG, BEST, WORST for key metrics."""
    if task == 'regression':
        metrics = ['rmse_oikan', 'r2_oikan', 'mape_oikan', 'rmse_elasticnet', 'r2_elasticnet', 'mape_elasticnet']
        best_worst = {
            'rmse_oikan': ('min', 'max'),
            'r2_oikan': ('max', 'min'),
            'mape_oikan': ('min', 'max'),
            'rmse_elasticnet': ('min', 'max'),
            'r2_elasticnet': ('max', 'min'),
            'mape_elasticnet': ('min', 'max')
        }
    else:  # classification
        metrics = ['accuracy_oikan', 'precision_oikan', 'f1_oikan', 'accuracy_elasticnet', 'precision_elasticnet', 'f1_elasticnet']
        best_worst = {metric: ('max', 'min') for metric in metrics}
    
    summary = {}
    for metric in metrics:
        avg = df[metric].mean()
        best = df[metric].agg(best_worst[metric][0])
        worst = df[metric].agg(best_worst[metric][1])
        summary[metric] = {'AVG': avg, 'BEST': best, 'WORST': worst}
    return summary

In [8]:
def save_summary_to_txt(summary, filename):
    """Save summary statistics to a .txt file."""
    with open(filename, 'w') as f:
        for metric, stats in summary.items():
            f.write(f"{metric}:\n")
            for key, value in stats.items():
                f.write(f"  {key}: {value:.4f}\n")
            f.write("\n")

In [9]:
print("Starting OIKAN Advanced Auto-Benchmarking based on PMLB Datasets...\n")

# Updated list of ~100 small-sized regression datasets
regression_datasets = [
    '1027_ESL', '1028_SWD', '1029_LEV', '1030_ERA', '1089_USCrime', '1096_FacultySalaries',
    '192_vineyard', '197_cpu_act', '201_pol', '210_cloud', '215_2dplanes', '218_house_8L',
    '225_puma8NH', '227_cpu_small', '228_elusage', '229_pwLinear', '230_machine_cpu',
    '294_satellite_image', '344_mv', '4544_GeographicalOriginalofMusic', '485_analcatdata_vehicle',
    '519_vineyard', '522_pm10', '523_analcatdata_neavote', '527_analcatdata_election2000',
    '529_pollen', '542_pollution', '547_no2', '556_analcatdata_apnea2', '557_analcatdata_apnea1',
    '559_bodyfat', '561_cpu', '562_cpu_small', '564_fried', '573_cpu_act', '574_house_16H',
    '579_fri_c0_250_5', '581_fri_c3_500_25', '582_fri_c1_1000_50', '583_fri_c1_1000_50',
    '586_fri_c3_1000_25', '587_fri_c3_1000_50', '588_fri_c4_1000_100', '589_fri_c2_1000_25',
    '590_fri_c0_1000_50', '591_fri_c1_100_10', '592_fri_c4_1000_25', '593_fri_c1_500_10',
    '594_fri_c2_1000_25', '595_fri_c0_1000_10', '596_fri_c2_500_25', '597_fri_c3_250_25',
    '598_fri_c4_1000_50', '599_fri_c2_1000_5', '601_fri_c1_500_5', '602_fri_c3_1000_10',
    '603_fri_c0_500_50', '604_fri_c4_500_25', '605_fri_c2_250_25', '606_fri_c4_1000_100',
    '607_fri_c4_500_100', '608_fri_c3_1000_5', '609_fri_c0_1000_5', '612_fri_c1_1000_5',
    '613_fri_c2_500_5', '615_fri_c4_250_25', '616_fri_c4_500_50', '617_fri_c3_500_5',
    '618_fri_c0_100_50', '620_fri_c1_1000_25', '621_fri_c0_100_10', '623_fri_c4_1000_10',
    '624_fri_c0_100_5', '625_fri_c0_500_10', '626_fri_c2_500_10', '627_fri_c1_500_50',
    '628_fri_c3_1000_100', '629_fri_c1_250_50', '630_fri_c3_500_50', '631_fri_c1_500_25',
    '632_fri_c3_250_50', '633_fri_c2_100_25', '634_fri_c2_1000_50', '635_fri_c0_250_10',
    '636_fri_c2_100_10', '637_fri_c3_500_10', '638_fri_c1_250_25', '639_fri_c2_250_50',
    '641_fri_c1_250_10', '642_fri_c3_250_10', '643_fri_c2_250_10', '644_fri_c4_250_50',
    '645_fri_c3_100_25', '646_fri_c3_100_10', '647_fri_c1_100_25', '648_fri_c4_100_25',
    '649_fri_c0_500_5', '650_fri_c0_250_50', '651_fri_c0_1000_25', '652_fri_c2_100_5'
]

# Updated list of ~100 small-sized classification datasets
classification_datasets = [
    'adult', 'mushroom', 'iris', 'wine', 'breast_cancer_wisconsin', 'tic_tac_toe',
    'analcatdata_authorship', 'analcatdata_fraud', 'penguins', 'monk3', 'balance_scale',
    'car_evaluation', 'credit_g', 'dermatology', 'glass', 'heart_c', 'ionosphere', 'kr_vs_kp',
    'lymphography', 'zoo', 'analcatdata_boxing1', 'analcatdata_boxing2', 'analcatdata_creditscore',
    'analcatdata_dmft', 'analcatdata_lawsuit', 'analcatdata_supreme', 'appendicitis', 'australian',
    'backache', 'biomed', 'bupa', 'clean1', 'clean2', 'coil2000', 'collins', 'confidence',
    'corral', 'crx', 'dis', 'ecoli', 'flags', 'grub_damage', 'haberman', 'hayes_roth', 'heart_h',
    'heart_statlog', 'hepatitis', 'horse_colic', 'hungarian', 'hypothyroid', 'labor', 'led7',
    'liver', 'lupus', 'machine', 'mfeat_factors', 'mfeat_fourier', 'mfeat_karhunen', 'mfeat_morphological',
    'mfeat_pixel', 'mfeat_zernike', 'monk1', 'monk2', 'movement_libras', 'new_thyroid', 'optdigits',
    'page_blocks', 'parity5', 'parity5+5', 'phoneme', 'pima', 'postoperative', 'prnn_crabs',
    'prnn_fglass', 'prnn_synth', 'ring', 'saheart', 'schizo', 'shuttle_landing', 'solar_flare_1',
    'solar_flare_2', 'sonar', 'spect', 'spectf', 'tae', 'threeOf9', 'tokenfile', 'trains',
    'tumor', 'twonorm', 'vehicle', 'veteran', 'vowel', 'wdbc', 'wine_quality_red', 'wine_quality_white',
    'wisconsin', 'yeast', 'zernike'
]

Starting OIKAN Advanced Auto-Benchmarking based on PMLB Datasets...



In [10]:
# Benchmark classification datasets
classification_results = []
for idx, dataset in enumerate(classification_datasets, 1):
    result = benchmark_classification_dataset(dataset, counter=idx, total=len(classification_datasets))
    if result:
        classification_results.append(result)
        if len(classification_results) == 60:
            break
classification_df = pd.DataFrame(classification_results)

/usr/local/lib/python3.11/dist-packages/oikan/model.py:290: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y, dtype=torch.float32))
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.750e+08, tolerance: 7.396e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.847e+07, tolerance: 1.199e+05
  model = cd_fast.enet_coordinate_descent(
/usr/l

Classification dataset: adult - completed (1 / 99) ✅


/usr/local/lib/python3.11/dist-packages/oikan/model.py:290: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y, dtype=torch.float32))
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.469e+05, tolerance: 1.519e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.572e+05, tolerance: 1.571e+03
  model = cd_fast.enet_coordinate_descent(
/usr/l

Classification dataset: mushroom - completed (2 / 99) ✅


/usr/local/lib/python3.11/dist-packages/oikan/model.py:290: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y, dtype=torch.float32))
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.956e+02, tolerance: 8.258e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.041e+01, tolerance: 1.227e+00
  model = cd_fast.enet_coordinate_descent(
/usr/l

...

Classification dataset: saheart - completed (77 / 99) ✅


/usr/local/lib/python3.11/dist-packages/oikan/model.py:290: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y, dtype=torch.float32))
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.357e+03, tolerance: 1.295e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.792e+03, tolerance: 5.080e+00
  model = cd_fast.enet_coordinate_descent(
/usr/l

Classification dataset: schizo - completed (78 / 99) ✅
Error benchmarking classification dataset shuttle_landing: Dataset not found in PMLB.
Error benchmarking classification dataset solar_flare_1: Dataset not found in PMLB.
Error benchmarking classification dataset solar_flare_2: Dataset not found in PMLB.


/usr/local/lib/python3.11/dist-packages/oikan/model.py:290: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y, dtype=torch.float32))


Classification dataset: sonar - completed (82 / 99) ✅


In [11]:
# Benchmark regression datasets
regression_results = []
for idx, dataset in enumerate(regression_datasets, 1):
    result = benchmark_regression_dataset(dataset, counter=idx, total=len(regression_datasets))
    if result:
        regression_results.append(result)
        if len(regression_results) == 60:
            break
regression_df = pd.DataFrame(regression_results)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.269e+02, tolerance: 1.122e+01
  model = cd_fast.enet_coordinate_descent(


Regression dataset: 1027_ESL - completed (1 / 100) ✅


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.135e+02, tolerance: 1.202e+01
  model = cd_fast.enet_coordinate_descent(


Regression dataset: 1028_SWD - completed (2 / 100) ✅


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.249e+00, tolerance: 2.539e+00
  model = cd_fast.enet_coordinate_descent(


...

Regression dataset: 562_cpu_small - completed (33 / 100) ✅
Regression dataset: 564_fried - completed (34 / 100) ✅


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.620e+06, tolerance: 2.940e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.729e+06, tolerance: 2.940e+05
  model = cd_fast.enet_coordinate_descent(


Regression dataset: 573_cpu_act - completed (35 / 100) ✅


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.370e+13, tolerance: 2.593e+10
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.289e+13, tolerance: 2.593e+10
  model = cd_fast.enet_coordinate_descent(


Regression dataset: 574_house_16H - completed (36 / 100) ✅
Regression dataset: 579_fri_c0_250_5 - completed (37 / 100) ✅
Regression dataset: 581_fri_c3_500_25 - completed (38 / 100) ✅
Error benchmarking regression dataset 582_fri_c1_1000_50: Dataset not found in PMLB.
Regression dataset: 583_fri_c1_1000_50 - completed (40 / 100) ✅
Regression dataset: 586_fri_c3_1000_25 - completed (41 / 100) ✅
Error benchmarking regression dataset 587_fri_c3_1000_50: Dataset not found in PMLB.
Regression dataset: 588_fri_c4_1000_100 - completed (43 / 100) ✅
Regression dataset: 589_fri_c2_1000_25 - completed (44 / 100) ✅
Regression dataset: 590_fri_c0_1000_50 - completed (45 / 100) ✅
Regression dataset: 591_fri_c1_100_10 - completed (46 / 100) ✅
Regression dataset: 592_fri_c4_1000_25 - completed (47 / 100) ✅
Error benchmarking regression dataset 593_fri_c1_500_10: Dataset not found in PMLB.
Error benchmarking regression dataset 594_fri_c2_1000_25: Dataset not found in PMLB.
Regression dataset: 595_fri_c

In [12]:
# Calculate and save summary statistics
if not regression_df.empty:
    regression_summary = calculate_summary_stats(regression_df, 'regression')
    save_summary_to_txt(regression_summary, 'regression_summary.txt')
if not classification_df.empty:
    classification_summary = calculate_summary_stats(classification_df, 'classification')
    save_summary_to_txt(classification_summary, 'classification_summary.txt')

In [13]:
# Display results
print("\nRegression Benchmark Results:")
print(regression_df.to_string(index=False))

print("\nClassification Benchmark Results:")
print(classification_df.to_string(index=False))


Regression Benchmark Results:
                         dataset device  num_features  num_augmented_rows  augmentation_factor  train_time  train_memory  predict_time  predict_memory   rmse_oikan    r2_oikan   mape_oikan  rmse_elasticnet  r2_elasticnet  mape_elasticnet
                        1027_ESL   cuda             4                3900                   10    3.339949      5.539176      0.000971        0.018075     0.507197    0.836351 8.205285e-02         0.781992       0.610985     1.404951e-01
                        1028_SWD   cuda            10                8000                   10    7.321452     29.553561      0.001886        0.048819     0.671735    0.306843 1.453503e-01         0.808703      -0.004647     1.895733e-01
                        1029_LEV   cuda             4                8000                   10    6.510589     11.306452      0.000842        0.028516     0.718013    0.396037 2.963707e+14         0.871962       0.109282     5.274147e+14
                 

In [14]:
# Save results to CSV files
regression_df.to_csv('regression_benchmark_results.csv', index=False)
classification_df.to_csv('classification_benchmark_results.csv', index=False)

print("\nBenchmarking completed. Results saved to 'regression_benchmark_results.csv' and 'classification_benchmark_results.csv'.")
print("Summary statistics saved to 'regression_summary.txt' and 'classification_summary.txt'.")


Benchmarking completed. Results saved to 'regression_benchmark_results.csv' and 'classification_benchmark_results.csv'.
Summary statistics saved to 'regression_summary.txt' and 'classification_summary.txt'.
